##### Datenverarbeitung des Global Population and Migration Dataset
##### Dieses Notebook lädt die heruntergeladenen Daten und bereitet sie für die Analyse auf.
##### 
##### Last Updated: 15.12.2024
##### ===============================================


#### **1. Imports und Datenladen**
#### ------------------------

In [46]:
import pandas as pd
import numpy as np
from pathlib import Path
import numpy as np

##### Daten laden

In [50]:
data_path = Path('../data')

# Zeige verfügbare Dateien
print("Verfügbare Dateien im data-Ordner:")
for file in data_path.glob('*.csv'):
    print(f"- {file.name}")

Verfügbare Dateien im data-Ordner:
- processed_world_pop_mig_186_countries.csv
- world_pop_mig_186_countries.csv


##### Datentypen anpassen

In [52]:
df['netMigration'] = df['netMigration'].astype(int)

#### **2. Länderbasierte Berechnungen**
# ----------------------------

In [54]:
# Aggregation nach Ländern
country_metrics = df.groupby(by="country").agg({
    'population': 'mean',
    'netMigration': 'sum',
    'population_in_millions': 'mean'
}).reset_index()

# Top/Bottom Migration
top_immigration = country_metrics.nlargest(10, 'netMigration')
top_emigration = country_metrics.nsmallest(10, 'netMigration')

#### **3. Zeitbasierte Berechnungen**
# --------------------------

In [55]:
yearly_metrics = df.groupby('year').agg({
    'population': 'sum',
    'netMigration': 'sum',
    'population_in_millions': 'sum'
}).reset_index()

#### **4. Aktuelle Bevölkerungszahlen (2023)**
# ----------------------------------

In [56]:
current_population = df[df['year'] == 2023].copy()
top_population_countries = current_population.nlargest(10, 'population')

#### **5. Migrations-Kategorisierung**
# --------------------------

In [57]:
migration_categories = pd.cut(df['netMigration'], 
                            bins=[float('-inf'), -1, 0, float('inf')],
                            labels=['Immigration', 'Neutral', 'Emigration'])
migration_distribution = migration_categories.value_counts()

In [ ]:
# 6. Regionale Statistiken
# ----------------------
# Migrations-Statistiken nach Kontinenten/Regionen (falls verfügbar)
if 'region' in df.columns:
    regional_stats = df.groupby('region').agg({
        'netMigration': ['sum', 'mean'],
        'population': ['mean', 'max']
    }).round(2)

# 7. Korrelationsanalyse
# --------------------
correlation_matrix = df.select_dtypes(include=[np.number]).corr()

# 8. Daten speichern
# ----------------
# Speichere alle berechneten Daten für Visualisierung
processed_data = {
    'country_metrics': country_metrics,
    'top_immigration': top_immigration,
    'top_emigration': top_emigration,
    'yearly_metrics': yearly_metrics,
    'top_population': top_population_countries,
    'migration_distribution': migration_distribution,
    'correlation_matrix': correlation_matrix
}

# Speichere die Daten als CSV
for name, data in processed_data.items():
    if isinstance(data, pd.DataFrame):
        data.to_csv(data_path / f'analyzed_{name}.csv', index=False)
    elif isinstance(data, pd.Series):
        data.to_frame().to_csv(data_path / f'analyzed_{name}.csv')

print("\nDatenverarbeitung abgeschlossen! Alle Kennzahlen wurden berechnet und gespeichert.")

# 9. Übersicht der Ergebnisse
# -------------------------
print("\nÜbersicht der wichtigsten Kennzahlen:")
print(f"Anzahl untersuchter Länder: {len(country_metrics)}")
print(f"Untersuchungszeitraum: {yearly_metrics['year'].min()} - {yearly_metrics['year'].max()}")
print("\nTop 3 Immigrationsländer:")
print(top_immigration[['country', 'netMigration']].head(3))
print("\nTop 3 Emigrationsländer:")
print(top_emigration[['country', 'netMigration']].head(3))